# HPVsim demonstration
In this tutorial we will walk through how to set up a simulation, define some parameters, and compare different simulations. Then we will learn how to use analyzers to extract more information than is provided by default.

Let's start by just running a super simple simulation:

In [ ]:
# take care of imports
import sciris as sc
import numpy as np
import pylab as pl
import hpvsim as hpv

In [ ]:
sim = hpv.Sim()
sim.run()
fig = sim.plot()

Now let's change the default parameters. I am going to increase the number of agents, specify start and end dates, genotypes, and location.

In [ ]:
base_pars = dict(
    n_agents=10e3,
    start=1980,
    end=2020,
    genotypes=[16, 18, 'hi5', 'ohr'],
    location='india',
    verbose=0.0
)
sim = hpv.Sim(pars=base_pars)
sim.run()
sim.plot()

Now let's investigate/test some of the assumptions of this model. 

In [ ]:
print(sim.pars['debut'])

In [ ]:
    scenarios = {
        'sexual_debut_15': {
            'name':'Sexual debut age 15',
            'pars': {
                
            }
        },
        'sexual_debut_17': {
            'name':'Sexual debut age 17',
            'pars': {
                'debut': dict(f=dict(dist='lognormal', par1=17, par2=2.),
                  m=dict(dist='lognormal', par1=17.6, par2=2.)),

            }
        },
        'sexual_debut_19': {
            'name':'Sexual debut age 19',
            'pars': {
                'debut': dict(f=dict(dist='lognormal', par1=19, par2=2.),
                  m=dict(dist='lognormal', par1=17.6, par2=2.)),

            }
        },
    }
    
    base_sim = hpv.Sim(pars=base_pars)
    metapars = {'n_runs': 3}
    scens = hpv.Scenarios(sim=base_sim, metapars=metapars, scenarios=scenarios)
    scens.run()

    print('Done running scenarios')
    

In [ ]:
to_plot = {
     'HPV incidence': ['hpv_incidence'],   
     'Age standardized cancer incidence (per 100,000 women)': ['asr_cancer_incidence'],
    'Cancer deaths per 100,000 women': ['cancer_mortality'],
 }
scens.plot(to_plot=to_plot)

Now, let's set up an *analyzer* to give us more information than is provided by default on the sexual network. HPVsim has several default analyzers you can use, or you can define a custom analyzer. Snapshots records and returns a copy of the people object on specified timepoints. This will work fine for our purposes!

In [ ]:
# First define analyzer
snap = hpv.snapshot(
    timepoints=['2020']
)

# Pass analyzer to the sim and re-run
sim = hpv.Sim(pars=base_pars, analyzers=[snap])
sim.run()

In [ ]:
# We are going to want to re-use this code, so let's turn it into a function

def plot_rship_counts(sim):
    # Create figure
    fig, axes = pl.subplots(ncols=3, figsize=(14, 10), sharey='col')

    # Extract the people snapshot
    people = sim.get_analyzer().snapshots[0]

    # Determine relationship types
    pship_types = sim.pars['layer_probs'].keys()
    n_pship_types = len(pship_types)

    # Keep track of number of relationships among those active
    rships = np.zeros((n_pship_types, len(people.age_bin_edges)))
    for lk, lkey in enumerate(pship_types):
        active_ages = people.age[(people.n_rships[lk,:] >= 1)]
        n_rships_active = people.n_rships[:,(people.n_rships[lk,:] >= 1)]
        age_bins_active = np.digitize(active_ages, bins=people.age_bin_edges) - 1

        all_ages = people.age
        n_rships_all = people.n_rships
        age_bins_all = np.digitize(all_ages, bins=people.age_bin_edges) - 1

        for ab in np.unique(age_bins_active):
            inds = age_bins_active==ab
            rships[lk,ab] = n_rships_active[lk,inds].sum()/len(hpv.true(inds))

        ax = axes[lk]
        yy = rships[lk,:]
        ax.bar(people.age_bin_edges, yy, width=3)
        ax.set_xlabel(f'Age')
        ax.set_title(f'Number of relationships, {lkey}')
        
    fig.tight_layout()
    pl.show()

# Now let's plot it
plot_rship_counts(sim)

Ok this looks like too many marriages, what can we do to fix it?
1. Increase the duration of marriages, so fewer dissolve and search for next partner
2. Decrease participation rates

Let's try both and see

In [ ]:
# First let's see what the default duration of partnership is set to
dur_pship = sim.pars['dur_pship']
print(dur_pship)

In [ ]:
longer_dur = sc.dcp(dur_pship)
longer_dur['m']['par1'] = 20
print(longer_dur)

In [ ]:
pars_longer_dur = sc.mergedicts(base_pars, {'dur_pship': longer_dur})
print(pars_longer_dur)

In [ ]:
# Now let's re-run simulation and check relationships

sim_longer_dur = hpv.Sim(pars=pars_longer_dur, analyzers=[snap])
sim_longer_dur.run()

plot_rship_counts(sim_longer_dur)

Ok this looks better, but let's try reduing participation at older ages too.

In [ ]:
# Let's see what the default participation rate is set to
# Extract the female participation rate among marital relationships
participation_rate = sc.dcp(sim.pars['layer_probs'])
print(participation_rate['m'][1,])

In [ ]:
# Let's try reducing the rate of participation 5-fold

participation_rate['m'][1,] /= 5
print(participation_rate['m'][1,])

In [ ]:
pars_lower_participation = sc.mergedicts(pars_longer_dur, {'layer_probs': participation_rate})
print(pars_lower_participation)

In [ ]:
# Now let's re-run simulation and check relationships

sim_lower_participation = hpv.Sim(pars=pars_lower_participation, analyzers=[snap])
sim_lower_participation.run()

plot_rship_counts(sim_lower_participation)